In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
train = pd.read_csv("train_QSVM.csv")
test = pd.read_csv("test_QSVM.csv")

In [11]:
# Separar características y etiquetas para conjunto de entrenamiento
X_train = train.drop(columns=['category'])
y_train = train['category']

# Separar características y etiquetas para conjunto de prueba
X_test = test.drop(columns=['category'])
y_test= test['category']

In [22]:
class QKTCallback:
    """Callback wrapper class."""

    def __init__(self) -> None:
        self._data = [[] for i in range(5)]

    def callback(self, x0, x1=None, x2=None, x3=None, x4=None):
        """
        Args:
            x0: number of function evaluations
            x1: the parameters
            x2: the function value
            x3: the stepsize
            x4: whether the step was accepted
        """
        self._data[0].append(x0)
        self._data[1].append(x1)
        self._data[2].append(x2)
        self._data[3].append(x3)
        self._data[4].append(x4)

    def get_callback_data(self):
        return self._data

    def clear_callback_data(self):
        self._data = [[] for i in range(5)]

In [24]:
from qiskit_ibm_runtime import Session, SamplerV2, EstimatorV2
from qiskit.providers.fake_provider import Fake127QPulseV1
from qiskit_ibm_runtime.fake_provider import FakeBrisbane 
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, QasmSimulator
#from qiskit_aer.primitives import SamplerV2

from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel, TrainableFidelityStatevectorKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer  
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.circuit import ParameterVector

training_params = ParameterVector("θ", 1)


num_qubits = X_train.shape[1]

backend = AerSimulator(method='statevector', device='GPU')
session = Session(backend=backend)
pass_manager = generate_preset_pass_manager(optimization_level=0, target=backend.target)

sampler = SamplerV2(mode=session)
fidelity = ComputeUncompute(sampler=sampler, pass_manager=pass_manager)

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=2)
qk = TrainableFidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map, training_parameters=training_params)

In [27]:
from qiskit_machine_learning.optimizers import SPSA


# Set up the optimizer
cb_qkt = QKTCallback()
spsa_opt = SPSA(maxiter=10, callback=cb_qkt.callback, learning_rate=0.05, perturbation=0.05)

# Instantiate a quantum kernel trainer.
qkt = QuantumKernelTrainer(
    quantum_kernel=qk, loss="svc_loss", optimizer=spsa_opt, initial_point=[np.pi / 2]
)

In [28]:
import time

start = time.time()
qk_results = qk_trainer.fit(X_train, y_train)
stop = time.time()

train_time = stop - start

optimized_kernel = qk_results.quantum_kernel
print(f"Training Time: {train_time}")
print(qk_results)

ValueError: could not broadcast input array from shape (16,) into shape (15,)

In [ ]:
from qiskit_machine_learning.algorithms import QSVC
import time

qsvc = QSVC(quantum_kernel=optimized_kernel, C=1, random_state=42)

start = time.time()
qsvc.fit(X_train, y_train)
stop = time.time()

train_time = stop - start

start = time.time()
y_pred = qsvc.predict(X_test)
stop = time.time()

test_time = stop - start

print(f"Training Time: {train_time}")
print(f"Testing Time: {test_time}")
print(f"Total Time: {train_time + test_time}")
print(f"QSVC training score: {qsvc.score(X_train, y_train)}")
print('Precision en test: ', accuracy_score(y_test, y_pred))